In [1]:
import seaborn as sns
import pickle 


In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_rows', 200)

In [4]:
year19_path_list=  ['turnstile_190504.txt', 'turnstile_190511.txt','turnstile_190518.txt','turnstile_190525.txt']
year18_path_list=  ['turnstile_180505.txt', 'turnstile_180512.txt','turnstile_180519.txt','turnstile_180526.txt']
year17_path_list=  ['turnstile_170506.txt', 'turnstile_170513.txt','turnstile_170520.txt','turnstile_170527.txt']

In [5]:
def load_data(list_of_datasets):
    year = pd.concat(map(lambda x: pd.read_csv(x,parse_dates=[['DATE','TIME']]), 
        list_of_datasets))
    return year

In [6]:
def preprocessing(dataframe):
    dataframe.reset_index(drop=True,inplace=True)
    dataframe.drop(['C/A','LINENAME','DIVISION', 'DESC'],axis=1,inplace = True)
    dataframe['UNIT/SCP']= dataframe['UNIT'] + dataframe['SCP'] #creating an ideantifier column 
    dataframe =  dataframe[(dataframe['DATE_TIME'].dt.month > 4)]
    dataframe = dataframe.sort_values(['STATION', 'UNIT/SCP','DATE_TIME'], ascending= True)
    return  dataframe
    

In [7]:
def calculate_entries_column(dataframe):
    dataframe.reset_index(drop=True,inplace=True)
    dataframe['ENTRIES_DIFF']=0
    for index, row in dataframe.iterrows():
        if index == 0:
            continue
        if row['UNIT/SCP'] == dataframe['UNIT/SCP'][index-1] and row['STATION'] == dataframe['STATION'][index-1] and row['ENTRIES'] > int(dataframe['ENTRIES'][index-1]):
            dataframe['ENTRIES_DIFF'][index] = row['ENTRIES'] - int(dataframe['ENTRIES'][index-1])
            
    return dataframe

In [8]:
def calculate_exits_column(dataframe):
    dataframe.columns=dataframe.columns.str.strip()
    dataframe.reset_index(drop=True,inplace=True)
    dataframe['EXITS_DIFF']=0
    for index, row in dataframe.iterrows():
        if index == 0:
            continue
        if row['UNIT/SCP'] == dataframe['UNIT/SCP'][index-1] and row['STATION'] == dataframe['STATION'][index-1] and row['EXITS'] > int(dataframe['EXITS'][index-1]):
            dataframe['EXITS_DIFF'][index] = row['EXITS'] - int(dataframe['EXITS'][index-1])
            
    return dataframe

In [9]:
def calculate_total_traffic(dataframe):
    dataframe['TOTAL_TRAFFIC']= dataframe['ENTRIES_DIFF'] + dataframe['EXITS_DIFF']
    return dataframe

In [10]:
def generate_stat(dataframe):
    year19_traffic_per_station = dataframe.groupby('STATION').TOTAL_TRAFFIC.sum()
    year19_traffic_per_station = pd.DataFrame(year19_traffic_per_station)
    sorted_traffic = year19_traffic_per_station.sort_values(by=['TOTAL_TRAFFIC'], ascending= False)
    return sorted_traffic

In [11]:
def pickle_df(dataframe,filename):
    with open(filename+".pickle","wb")as to_write:
        pickle.dump(dataframe,to_write)

In [12]:
def read_pickle(file_name):
    with open(file_name, 'rb') as f:
        dataframe = pickle.load(f)
        return dataframe

In [13]:
def init_dataframe(years_list):
    year=load_data(years_list)
    year = preprocessing(year)
    year = calculate_entries_column(year)
    year = calculate_exits_column(year)
    return year 

In [14]:
def graph_stats(stats):
    stats = stats.head(5)
    stats = stats.reset_index()
    stats.columns
    sns.barplot(x=stats['TOTAL_TRAFFIC'], y=stats['STATION'])

In [15]:
year19_pickle = read_pickle('MTA_19.pickle')

In [ ]:
year19_pickle

In [16]:
median=year19_pickle.groupby('STATION').median()
Mediandf = pd.DataFrame(median) 

In [17]:
Mediandf.drop(['ENTRIES','EXITS','ENTRIES_DIFF', 'EXITS_DIFF'],axis=1,inplace = True)


In [ ]:
Mediandf.head()

In [18]:
Mediandf.reset_index()['TOTAL_TRAFFIC'].head()

0    347.0
1    527.0
2    582.5
3     22.0
4    436.0
Name: TOTAL_TRAFFIC, dtype: float64

In [19]:
year19= pd.merge(year19_pickle,Mediandf.reset_index()[['TOTAL_TRAFFIC','STATION']],on='STATION')
year19.columns=year19.columns.str.replace("TOTAL_TRAFFIC_y",'TOTAL_TRAFFIC_MED')
year19.columns=year19.columns.str.replace("TOTAL_TRAFFIC_x",'TOTAL_TRAFFIC')

year19.head()

,DATE_TIME,UNIT,SCP,STATION,ENTRIES,EXITS,UNIT/SCP,ENTRIES_DIFF,EXITS_DIFF,TOTAL_TRAFFIC,TOTAL_TRAFFIC_MED
0,2019-05-01 00:00:00,R248,00-00-00,1 AV,14654538,16381430,R24800-00-00,0,0,0,347.0
1,2019-05-01 04:00:00,R248,00-00-00,1 AV,14654538,16381436,R24800-00-00,0,6,6,347.0
2,2019-05-01 08:00:00,R248,00-00-00,1 AV,14654889,16382236,R24800-00-00,351,800,1151,347.0
3,2019-05-01 12:00:00,R248,00-00-00,1 AV,14656102,16383435,R24800-00-00,1213,1199,2412,347.0
4,2019-05-01 16:00:00,R248,00-00-00,1 AV,14656924,16384214,R24800-00-00,822,779,1601,347.0


In [ ]:
year19_pickle

In [20]:
Q1=year19_pickle.groupby('STATION').quantile(q=0.25)
q1df = pd.DataFrame(Q1) 
q1df

0.25,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF,TOTAL_TRAFFIC
STATION,,,,,
1 AV,5.565480e+05,1045804.50,7.00,14.00,61.75
103 ST,4.323436e+06,3559117.00,86.00,62.00,188.00
103 ST-CORONA,3.987503e+06,1644733.50,89.00,66.00,279.50
104 ST,3.057604e+06,1034438.00,0.00,0.00,1.00
110 ST,4.661174e+06,1666844.00,67.00,75.50,156.50
111 ST,7.967990e+05,287326.00,8.00,4.00,16.50
116 ST,1.829575e+05,116362.75,60.00,43.00,111.00
116 ST-COLUMBIA,5.321377e+06,2630559.00,88.00,38.25,169.50
121 ST,8.271988e+05,299128.50,3.00,4.00,9.00


In [21]:
q1df.drop(['ENTRIES','EXITS','ENTRIES_DIFF', 'EXITS_DIFF'],axis=1,inplace = True)


In [22]:
q1df

0.25,TOTAL_TRAFFIC
STATION,
1 AV,61.75
103 ST,188.00
103 ST-CORONA,279.50
104 ST,1.00
110 ST,156.50
111 ST,16.50
116 ST,111.00
116 ST-COLUMBIA,169.50
121 ST,9.00


In [23]:
Q3=year19_pickle.groupby('STATION').quantile(q=0.75)
q3df = pd.DataFrame(Q3) 

In [24]:
q3df.drop(['ENTRIES','EXITS','ENTRIES_DIFF', 'EXITS_DIFF'],axis=1,inplace = True)
q3df

0.75,TOTAL_TRAFFIC
STATION,
1 AV,817.00
103 ST,796.00
103 ST-CORONA,794.00
104 ST,134.00
110 ST,719.50
111 ST,502.50
116 ST,534.00
116 ST-COLUMBIA,785.75
121 ST,155.25


In [25]:
IQR = pd.DataFrame(q3df['TOTAL_TRAFFIC'] - q1df['TOTAL_TRAFFIC'])

In [26]:
IQR

,TOTAL_TRAFFIC
STATION,
1 AV,755.25
103 ST,608.00
103 ST-CORONA,514.50
104 ST,133.00
110 ST,563.00
111 ST,486.00
116 ST,423.00
116 ST-COLUMBIA,616.25
121 ST,146.25


In [27]:
Mediandf

,TOTAL_TRAFFIC
STATION,
1 AV,347.0
103 ST,527.0
103 ST-CORONA,582.5
104 ST,22.0
110 ST,436.0
111 ST,105.0
116 ST,329.0
116 ST-COLUMBIA,426.5
121 ST,97.0


In [28]:
upper = pd.DataFrame(Mediandf['TOTAL_TRAFFIC'] + 1.5 * IQR['TOTAL_TRAFFIC'])

In [29]:
upper

,TOTAL_TRAFFIC
STATION,
1 AV,1479.875
103 ST,1439.000
103 ST-CORONA,1354.250
104 ST,221.500
110 ST,1280.500
111 ST,834.000
116 ST,963.500
116 ST-COLUMBIA,1350.875
121 ST,316.375


In [30]:
lower= pd.DataFrame(Mediandf['TOTAL_TRAFFIC'] - 1.5 * IQR['TOTAL_TRAFFIC'])

In [31]:
lower


,TOTAL_TRAFFIC
STATION,
1 AV,-785.875
103 ST,-385.000
103 ST-CORONA,-189.250
104 ST,-177.500
110 ST,-408.500
111 ST,-624.000
116 ST,-305.500
116 ST-COLUMBIA,-497.875
121 ST,-122.375


In [32]:
year19

,DATE_TIME,UNIT,SCP,STATION,ENTRIES,EXITS,UNIT/SCP,ENTRIES_DIFF,EXITS_DIFF,TOTAL_TRAFFIC,TOTAL_TRAFFIC_MED
0,2019-05-01 00:00:00,R248,00-00-00,1 AV,14654538,16381430,R24800-00-00,0,0,0,347.0
1,2019-05-01 04:00:00,R248,00-00-00,1 AV,14654538,16381436,R24800-00-00,0,6,6,347.0
2,2019-05-01 08:00:00,R248,00-00-00,1 AV,14654889,16382236,R24800-00-00,351,800,1151,347.0
3,2019-05-01 12:00:00,R248,00-00-00,1 AV,14656102,16383435,R24800-00-00,1213,1199,2412,347.0
4,2019-05-01 16:00:00,R248,00-00-00,1 AV,14656924,16384214,R24800-00-00,822,779,1601,347.0
5,2019-05-01 20:00:00,R248,00-00-00,1 AV,14657978,16385015,R24800-00-00,1054,801,1855,347.0
6,2019-05-02 00:00:00,R248,00-00-00,1 AV,14658165,16385220,R24800-00-00,187,205,392,347.0
7,2019-05-02 04:00:00,R248,00-00-00,1 AV,14658165,16385220,R24800-00-00,0,0,0,347.0
8,2019-05-02 08:00:00,R248,00-00-00,1 AV,14658505,16386045,R24800-00-00,340,825,1165,347.0
9,2019-05-02 09:45:01,R248,00-00-00,1 AV,14659386,16386724,R24800-00-00,881,679,1560,347.0


In [33]:
year19_up = pd.merge(year19,upper.reset_index()[['TOTAL_TRAFFIC','STATION']],on='STATION')
year19_up.columns=year19_up.columns.str.replace("TOTAL_TRAFFIC_x",'TOTAL_TRAFFIC')
year19_up.columns=year19_up.columns.str.replace("TOTAL_TRAFFIC_y",'TOTAL_TRAFFIC_UPPER')

year19_up.head()


,DATE_TIME,UNIT,SCP,STATION,ENTRIES,EXITS,UNIT/SCP,ENTRIES_DIFF,EXITS_DIFF,TOTAL_TRAFFIC,TOTAL_TRAFFIC_MED,TOTAL_TRAFFIC_UPPER
0,2019-05-01 00:00:00,R248,00-00-00,1 AV,14654538,16381430,R24800-00-00,0,0,0,347.0,1479.875
1,2019-05-01 04:00:00,R248,00-00-00,1 AV,14654538,16381436,R24800-00-00,0,6,6,347.0,1479.875
2,2019-05-01 08:00:00,R248,00-00-00,1 AV,14654889,16382236,R24800-00-00,351,800,1151,347.0,1479.875
3,2019-05-01 12:00:00,R248,00-00-00,1 AV,14656102,16383435,R24800-00-00,1213,1199,2412,347.0,1479.875
4,2019-05-01 16:00:00,R248,00-00-00,1 AV,14656924,16384214,R24800-00-00,822,779,1601,347.0,1479.875


In [34]:
year19_up

,DATE_TIME,UNIT,SCP,STATION,ENTRIES,EXITS,UNIT/SCP,ENTRIES_DIFF,EXITS_DIFF,TOTAL_TRAFFIC,TOTAL_TRAFFIC_MED,TOTAL_TRAFFIC_UPPER
0,2019-05-01 00:00:00,R248,00-00-00,1 AV,14654538,16381430,R24800-00-00,0,0,0,347.0,1479.875
1,2019-05-01 04:00:00,R248,00-00-00,1 AV,14654538,16381436,R24800-00-00,0,6,6,347.0,1479.875
2,2019-05-01 08:00:00,R248,00-00-00,1 AV,14654889,16382236,R24800-00-00,351,800,1151,347.0,1479.875
3,2019-05-01 12:00:00,R248,00-00-00,1 AV,14656102,16383435,R24800-00-00,1213,1199,2412,347.0,1479.875
4,2019-05-01 16:00:00,R248,00-00-00,1 AV,14656924,16384214,R24800-00-00,822,779,1601,347.0,1479.875
5,2019-05-01 20:00:00,R248,00-00-00,1 AV,14657978,16385015,R24800-00-00,1054,801,1855,347.0,1479.875
6,2019-05-02 00:00:00,R248,00-00-00,1 AV,14658165,16385220,R24800-00-00,187,205,392,347.0,1479.875
7,2019-05-02 04:00:00,R248,00-00-00,1 AV,14658165,16385220,R24800-00-00,0,0,0,347.0,1479.875
8,2019-05-02 08:00:00,R248,00-00-00,1 AV,14658505,16386045,R24800-00-00,340,825,1165,347.0,1479.875
9,2019-05-02 09:45:01,R248,00-00-00,1 AV,14659386,16386724,R24800-00-00,881,679,1560,347.0,1479.875


In [36]:
year19_up.loc[year19_up.TOTAL_TRAFFIC > year19_up.TOTAL_TRAFFIC_UPPER,"TOTAL_TRAFFIC"]

3         2412
4         1601
5         1855
9         1560
11        1703
12        2213
16        1705
18        1708
19        1960
35        2292
36        1627
37        1773
41        2126
44        1661
45        1974
49        2384
50        1585
51        1899
55        2391
56        1715
57        1998
61        2464
62        1777
63        1958
81        1947
82        1573
83        1561
87        2329
88        1652
89        1883
96        1892
97        1689
98        1940
104       1925
105       1618
106       2002
112       1634
113       1764
114       1985
130       2322
131       1606
132       1839
138       1589
139       1600
140       1978
144       2438
145       1663
146       1855
152       2261
153       1703
154       1938
158       2233
159       1827
160       1933
164       1871
166       1545
180       1588
196       1742
202       1523
206       1482
210       1909
212       1491
216       1839
218       1639
222       1727
224       1492
242       

In [37]:
year19_up.loc[year19_up.TOTAL_TRAFFIC > year19_up.TOTAL_TRAFFIC_UPPER,"TOTAL_TRAFFIC"] =year19_up.loc[year19_up.TOTAL_TRAFFIC > year19_up.TOTAL_TRAFFIC_UPPER,"TOTAL_TRAFFIC_MED"]

In [38]:
year19_up.loc[year19_up.TOTAL_TRAFFIC > year19_up.TOTAL_TRAFFIC_UPPER,"TOTAL_TRAFFIC"]

Series([], Name: TOTAL_TRAFFIC, dtype: float64)

In [ ]:
#=year19_up.loc[year19_up.TOTAL_TRAFFIC_x > year19_up.TOTAL_TRAFFIC_UPPER,"TOTAL_TRAFFIC_MED"]

In [ ]:
year19_pickle

In [ ]:
stats = generate_stat(year19_pickle)

In [ ]:
graph_stats(stats)

In [ ]:
# year18 = init_dataframe(year18_path_list)

In [ ]:
# pickle_df(year18,'MTA_18' )

In [ ]:
year18_pickle = read_pickle('MTA_18.pickle')

In [ ]:
# year17 = init_dataframe(year17_path_list)

In [ ]:
# pickle_df(year17,'MTA_17')

In [ ]:
year17_pickle = read_pickle('MTA_17.pickle')

In [ ]:
year17_pickle